In [1]:
"""
import tarfile

tar = tarfile.open('comm-f2f-Resistance-network.tar.gz', "r:gz")
tar.extractall()
tar.close()
"""

'\nimport tarfile\n\ntar = tarfile.open(\'comm-f2f-Resistance-network.tar.gz\', "r:gz")\ntar.extractall()\ntar.close()\n'

In [71]:
%cd "../data"
%ls

/Users/shy/Projects/CSCI5527/project/data
network_list.csv  node_role.csv  processed_networks/  raw_networks/


Code taken from the README file

In [72]:
import numpy as np
import pandas as pd

src = 'raw_networks' # root dir of data
meta = pd.read_csv('network_list.csv')
roles = pd.read_csv('node_role.csv')
THRESHOLD = 0.7

def load_game(game, is_weighted):
    # N is the number of players in the game
    # load csv data
    # unweighted and weighted networks can be loaded exactly the same way
    # below shows the loader for weighted networks
    
    df_network = pd.read_csv(f"{src}/network{game}_weighted.csv" if is_weighted else f"{src}/network{game}.csv", index_col=0)

    # for dense repr
    #df_network = pd.read_csv(f"{src}/network{game}_weighted.csv", index_col=0)
    
    # get meta NUMBER_OF_PARTICIPANTS by game NETWORK id
    N = meta.loc[game, "NUMBER_OF_PARTICIPANTS"]

    # T is number of timestamps (10 frames)
    T = len(df_network)
    # load VFOA network to T x N x (N+1) array
    # vfoa[t, n, i] is the probability of player n+1 looking at object i at time t
    # i: 0 - laptop, 1 - player 1, 2 - player 2, ..., N - player N
    vfoa = np.reshape(df_network.values, (T, N, N + 1))

    # print information
    print(f"network id:{game}\t length(x 1/3 second): {T}\t num of players: {N}")
    return vfoa

def process_game(game, game_id, feature_type='constant'):
    records = []

    t, n1, n2 = game.shape

    # this is 0-indexed while nodes are 1-indexed
    game_roles = {i: roles[
                (roles["node"] == f"P{i+1}") & (roles["ID"] == game_id)
            ]["role"].values[0]-1 for i in range(n1)}

    records = []
    for timestep, frame in enumerate(game):
        #print(f"timestep {timestep} {frame}")

        # get max index of each row
        max_index = np.argmax(frame, axis=1)
        
        # get max value of each row
        max_value = np.max(frame, axis=1)
        
        for i, item_id in enumerate(max_index):
            if max_value[i] < THRESHOLD:
                continue
            # get the role of user i in game game_id
            role = game_roles[i] 
    
            features = []
            if feature_type == 'constant':
                features.append(1)
            elif feature_type == 'prob':
                features.append(max_value[i])
            elif feature_type == 'labels':
                pass
            elif feature_type == 'probabilities':
                pass #TODO
            # record: user_id, item_id, timestamp, state_label, (empty) feature
            records.append([game_id * 10 + i + 1, game_id * 10 + item_id, timestep, role] + features)
    
    return records

Flatten the adjacency matrix so that each row represents the gaze vector of the user.  
Convert this into the wikipedia format and save as CSV 

### Create Single Network CSVs

In [30]:
for _, meta_row in meta.iterrows():
    game_id = meta_row['NETWORK']
    if len(roles[roles['ID'] == game_id]) == 0:
        print(f"No roles for game {game_id}")
        continue
        
    game = load_game(game_id, True)
    
    records = process_game(game, game_id, feature_type='prob')

    
    game_df = pd.DataFrame(records, columns=['user_id', 'item_id', 'timestamp', 'state_label', 'comma_separated_list_of_features'])
    game_df.to_csv(f"./network_processed/network_{game_id}.csv", index=False)

network id:0	 length(x 1/3 second): 7323	 num of players: 7
[0.534 0.503 0.533 0.377 0.357 0.15  0.172]


NameError: name 'featuers' is not defined

### Create All Network CSV

In [73]:
records = []
for _, meta_row in meta.iterrows():
    game_id = meta_row['NETWORK']
    if len(roles[roles['ID'] == game_id]) == 0:
        print(f"No roles for game {game_id}")
        continue
        
    game = load_game(game_id, True)

    records += process_game(game, game_id, feature_type='prob')

records.sort(key=lambda x: x[2]) # sort by timestep

game_df = pd.DataFrame(records, columns=['user_id', 'item_id', 'timestamp', 'state_label', 'comma_separated_list_of_features'])
game_df.to_csv(f"./processed_networks/networks_all.csv", index=False)

network id:0	 length(x 1/3 second): 7323	 num of players: 7
No roles for game 1
network id:2	 length(x 1/3 second): 7533	 num of players: 7
network id:3	 length(x 1/3 second): 6264	 num of players: 7
network id:4	 length(x 1/3 second): 7323	 num of players: 8
network id:5	 length(x 1/3 second): 9651	 num of players: 8
network id:6	 length(x 1/3 second): 7260	 num of players: 7
network id:7	 length(x 1/3 second): 7245	 num of players: 6
network id:8	 length(x 1/3 second): 6243	 num of players: 7
No roles for game 9
network id:10	 length(x 1/3 second): 6681	 num of players: 7
network id:11	 length(x 1/3 second): 5415	 num of players: 7
network id:12	 length(x 1/3 second): 5571	 num of players: 8
network id:13	 length(x 1/3 second): 7173	 num of players: 7
network id:14	 length(x 1/3 second): 9429	 num of players: 8
network id:15	 length(x 1/3 second): 5466	 num of players: 6
network id:16	 length(x 1/3 second): 6678	 num of players: 6
network id:17	 length(x 1/3 second): 3054	 num of pla

## Exploration

In [68]:
sorted(game_df['comma_separated_list_of_features'].unique())
len(game_df)

317325